In [ ]:
#FINALLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL
import sqlite3

# Connect to the database
conn = sqlite3.connect('customer.db')
cursor = conn.cursor()

# Create tables if they don't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        employee_id INTEGER PRIMARY KEY,
        username TEXT,
        password TEXT
    )
''')


cursor.execute('''
    CREATE TABLE IF NOT EXISTS branches (
        branch_id INTEGER PRIMARY KEY,
        branch_name TEXT,
        city TEXT,
        assets REAL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS loans (
        loan_no INTEGER PRIMARY KEY,
        cust_id INTEGER,
        branch_id INTEGER,
        loan_amount REAL,
        FOREIGN KEY (cust_id) REFERENCES customers (cust_id),
        FOREIGN KEY (branch_id) REFERENCES branches (branch_id)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS transactions (
        trans_id INTEGER PRIMARY KEY,
        acc_no INTEGER,
        trans_date TEXT,
        transaction_mode TEXT,
        amount REAL,
        FOREIGN KEY (acc_no) REFERENCES accounts (acc_no)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS accounts (
        acc_no INTEGER PRIMARY KEY,
        cust_id INTEGER,
        branch_id INTEGER,
        balance REAL,
        acc_type TEXT,
        status TEXT,
        FOREIGN KEY (cust_id) REFERENCES customers (cust_id),
        FOREIGN KEY (branch_id) REFERENCES branches (branch_id)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS customers (
        cust_id INTEGER PRIMARY KEY,
        cust_name TEXT,
        city TEXT,
        gender TEXT,
        dob TEXT,
        occupation TEXT
    )
''')

# Function to create a new customer
def create_customer(name, city, gender, dob, occupation):
    cursor.execute('''
        INSERT INTO customers (cust_name, city, gender, dob, occupation)
        VALUES (?, ?, ?, ?, ?)
    ''', (name, city, gender, dob, occupation))
    conn.commit()
    print("Customer created successfully!")

# Function to create a new branch
def create_branch(branch_id, branch_name, city, assets):
    cursor.execute('''
        INSERT INTO branches (branch_id, branch_name, city, assets)
        VALUES (?, ?, ?, ?)
    ''', (branch_id, branch_name, city, assets))
    conn.commit()
    print("Branch created successfully!")

# Function to create a new account
def create_account(cust_id, branch_id, balance, acc_type, status):
    cursor.execute('SELECT cust_name FROM customers WHERE cust_id = ?', (cust_id,))
    customer = cursor.fetchone()
    if not customer:
        print("Customer not found!")
        return

    customer_name = customer[0]

    # Generate account number
    cursor.execute('SELECT MAX(acc_no) FROM accounts')
    result = cursor.fetchone()
    if result[0] is not None:
        last_acc_no = result[0]
        new_acc_no = last_acc_no + 1
    else:
        new_acc_no = 1

    cursor.execute('''
        INSERT INTO accounts (acc_no, cust_id, branch_id, balance, acc_type, status)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (new_acc_no, cust_id, branch_id, balance, acc_type, status))
    conn.commit()
    print("Account created successfully with account number:", new_acc_no)

# Function to create a new loan
def create_loan(loan_no, cust_id, branch_id, loan_amount):
    cursor.execute('SELECT * FROM accounts WHERE acc_no = ?', (acc_no,))
    account = cursor.fetchone()
    if not account:
        print("Account not found!")
        return

    # Update account balance based on transaction type
    if transaction_mode == 'credit':
        new_balance = account[4] + amount
    elif transaction_mode == 'debit':
        new_balance = account[4] - amount
    else:
        print("Invalid transaction mode!")
        return

    cursor.execute('UPDATE accounts SET balance = ? WHERE acc_no = ?', (new_balance, acc_no))
def make_transaction(acc_no, trans_date, trans_mode, amount):
    # Record the transaction
    cursor.execute('INSERT INTO transactions (acc_no, trans_date, transaction_mode, amount) VALUES (?, ?, ?, ?)', (acc_no, trans_date, trans_mode, amount))

    conn.commit()
    print("Transaction successful!")

# Function to get customer information
def get_customer_info(cust_id):
    cursor.execute('SELECT * FROM customers WHERE cust_id = ?', (cust_id,))
    customer = cursor.fetchone()
    if not customer:
        print("Customer not found!")
        return

    print("Customer ID:", customer[0])
    print("Name:", customer[1])
    print("City:", customer[2])
    print("Gender:", customer[3])
    print("Date of Birth:", customer[4])
    print("Occupation:", customer[5])

# Main menu
def main_menu():
    while True:
        print("\n==== Bank Management System ====")
        print("1. Create a new customer")
        print("2. Create a new branch")
        print("3. Create a new account")
        print("4. Create a new loan")
        print("5. Make a transaction")
        print("6. Get customer information")
        print("7. Exit")
        choice = input("Enter your choice (1-7): ")

        if choice == '1':
            name = input("Enter customer name: ")
            city = input("Enter customer city: ")
            gender = input("Enter customer gender: ")
            dob = input("Enter customer date of birth (YYYY-MM-DD): ")
            occupation = input("Enter customer occupation: ")
            create_customer(name, city, gender, dob, occupation)
        elif choice == '2':
            branch_id = input("Enter branch ID: ")
            branch_name = input("Enter branch name: ")
            city = input("Enter city: ")
            assets = float(input("Enter assets: "))
            create_branch(branch_id, branch_name, city, assets)
        elif choice == '3':
            cust_id = int(input("Enter customer ID: "))
            branch_id = int(input("Enter branch ID: "))
            balance = 0
            while balance < 2000:
                balance = float(input("Enter account balance (minimum 2000): "))
                if balance < 2000:
                    print("Minimum balance should be 2000!")
            acc_type = input("Enter account type: ")
            status = input("Enter account status: ")
            create_account(cust_id, branch_id, balance, acc_type, status)
        elif choice == '4':
            loan_no = input("Enter loan number: ")
            cust_id = int(input("Enter customer ID: "))
            branch_id = int(input("Enter branch ID: "))
            loan_amount = float(input("Enter loan amount: "))
            create_loan(loan_no, cust_id, branch_id, loan_amount)
        elif choice == '5':
            acc_no = int(input("Enter account number: "))
            trans_date = input("Enter transaction date (YYYY-MM-DD): ")
            trans_mode = input("Enter transaction mode (credit/debit): ")
            amount = float(input("Enter transaction amount: "))
            make_transaction(acc_no, trans_date, trans_mode, amount)
        elif choice == '6':
            cust_id = int(input("Enter customer ID: "))
            get_customer_info(cust_id)
        elif choice == '7':
            break
        else:
            print("Invalid choice. Please try again.")

# Run the main menu
main_menu()

# Close the database connection
conn.close()


In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('customer.db')
cursor = conn.cursor()

# Fetch and print data from the customers table
print('Customers Info:')
cursor.execute('SELECT * FROM customers')
rows = cursor.fetchall()
for row in rows:
    print(row)

print('------------------------------------------')

# Fetch and print data from the branches table
print('Branches Info:')
cursor.execute('SELECT * FROM branches')
rows = cursor.fetchall()
for row in rows:
    print(row)

print('------------------------------------------')

# Fetch and print data from the accounts table
print('Accounts Info:')
cursor.execute('SELECT * FROM accounts')
rows = cursor.fetchall()
for row in rows:
    print(row)

print('------------------------------------------')

# Fetch and print data from the transactions table
print('Transactions Info:')
cursor.execute('SELECT * FROM transactions')
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the database connection
conn.close()


In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('employee.db')
cursor = conn.cursor()

# Create tables if they don't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        employee_id INTEGER PRIMARY KEY,
        username TEXT,
        password TEXT
    )
''')





def authenticate(username, password):
    cursor.execute('SELECT * FROM employees WHERE username = ? AND password = ?', (username, password))
    user = cursor.fetchone()
    if user:
        return True
    else:
        return False

# Function to create a new user
def create_employee(username, password):
    cursor.execute('SELECT * FROM employees  WHERE username = ?', (username,))
    user = cursor.fetchone()
    if user:
        print("Username already exists. Please choose a different username.")
        return
    cursor.execute('INSERT INTO employees (username, password) VALUES (?, ?)', (username, password))
    conn.commit()
    print("User created successfully!")


# Main menu
def main_menu(logged_in_user):
    while True:
        print("\n==== Employees Management System ====")
        if not logged_in_user:
            print("1. Register a new Employee")
            print("2. Login")
        
        print("3. Exit")
        choice = input("Enter your choice (1-3): ")

        if not logged_in_user:
            if choice == '1':
                username = input("Enter a username: ")
                password = input("Enter a password: ")
                create_employee(username, password)
            elif choice == '2':
                username = input("Enter your username: ")
                password = input("Enter your password: ")

                if authenticate(username, password):
                    logged_in_user = username
                    print("Authentication successful!")
                else:
                    print("Authentication failed. Please try again.")
            elif choice == '3':
                break
            else:
                print("Invalid choice. Please try again.")

       

# Run the main menu with logged_in_user as None
main_menu(None)

# Close the database connection
conn.close()

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('employee.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM employees')
rows=cursor.fetchall()
for i in rows:
    print(i)